# Training data extraction

This notebook runs the pose extraction for the training datasets.

1. Downloads required packages

2. Performs inference on training frames for post-hoc training poses

3. Performs inference on training frames for edge training poses

Dependencies: The image frames must be in the directory above root (../) relative to this notebook. 

In [ ]:
from typing import Literal

FEATURE_EXTRACTOR: Literal['edge', 'post-hoc'] = 'post-hoc'

In [ ]:
! pip install ultralytics pandas torch-hd torch

# Post Hoc Pose estimations

The post hoc pose estimation uses a larger model (longer run time)

In [ ]:
from src.estimate_poses import estimate_train_vids
from time import time
from ultralytics import YOLO

if FEATURE_EXTRACTOR == 'edge':
    model = YOLO("../yolo11s-pose_openvino_model/", task="pose")
else:
    model = YOLO("yolo11m-pose", task="pose")

start = time()
estimate_train_vids("../Avenue Dataset/training_videos", model, FEATURE_EXTRACTOR)
end = time()

print(f'Poses detected in {end-start}s')

# Edge poses

The edge system uses a quantised model, the first cell downloads and quantises a model, the second runs the inference pipeline.

In [ ]:
from ultralytics import YOLO 

# Load a model
model = YOLO("yolo11s-pose.pt")

# Export the model to OpenVINO format
model.export(format="openvino", dynamic=True) 

In [ ]:
from src.estimate_poses import estimate_train_vids
from time import time
from ultralytics import YOLO


FEATURE_EXTRACTOR = 'edge'

if FEATURE_EXTRACTOR == 'edge':
    model = YOLO("../yolo11s-pose_openvino_model/", task="pose")
else:
    model = YOLO("yolo11m-pose", task="pose")

start = time()
estimate_train_vids("../Avenue Dataset/training_videos", model, FEATURE_EXTRACTOR)
end = time()

print(f'Poses detected in {end-start}s')

## Infer on test frames: Post-hoc system only

The post-hoc system does not run on edge compute so the inference is done here.

The time taken here is used in the results when considering latency for the post-hoc system.

Note: For efficiency, switch `plot` to `False` for interpretability keep as it is, as this will output pose estimations every ten frames.

In [ ]:
from src.estimate_poses import estimate_train_vids
from time import time
from ultralytics import YOLO
FEATURE_EXTRACTOR = 'post-hoc'
if FEATURE_EXTRACTOR == 'edge':
    model = YOLO("../yolo11s-pose_openvino_model/", task="pose")
else:
    model = YOLO("yolo11m-pose", task="pose")

start = time()
estimate_train_vids("../Avenue Dataset/testing_videos", model, FEATURE_EXTRACTOR, plot=True)
end = time()

print(f'Poses detected in {end-start}s')